In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('firebase-svc-account-key.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

# 문서 필드
```bash
cost_per_hour: 0.87389
cpu: 8
gpu: null
name: "c3.2xlarge"
ram: 15
region: "ap-northeast-2"
vendor: "AWS"
```

In [8]:
gemini_result = {'gcp': {'instance': {'cloud_provider': 'GCP',
   'name': 'f1-micro',
   'cpu': 1,
   'ram': 0.6,
   'storage': 30,
   'gpu': 'None'},
  'power_consumption': {'monthly': '13.14 kWh', 'hourly': '0.018 kWh'},
  'carbon_footprint': {'monthly': '5.69 kg CO2', 'hourly': '0.008 kg CO2'},
  'description': "This application is a simple TODO backend application with minimal resource requirements. Thus, a small instance like the f1-micro should be sufficient to handle normal traffic. It comes with 1 vCPU and 0.6 GB of RAM. We'll assume 30GB storage is sufficient. The estimated power consumption and carbon footprint are based on Google Cloud's sustainability calculator, assuming the instance runs 24/7 in the us-central1 region."},
 'aws': {'instance': {'cloud_provider': 'AWS',
   'name': 't4g.nano',
   'cpu': 2,
   'ram': 0.5,
   'storage': 30,
   'gpu': 'None'},
  'power_consumption': {'monthly': '8.76 kWh', 'hourly': '0.012 kWh'},
  'carbon_footprint': {'monthly': '3.77 kg CO2', 'hourly': '0.005 kg CO2'},
  'description': "For AWS, we'll opt for the t4g.nano instance, offering 2 vCPUs and 0.5 GB of RAM, which is suitable for this lightweight application. We'll assume 30GB storage is sufficient. Power consumption and carbon footprint estimations are derived from AWS's carbon footprint tools, considering 24/7 operation in the us-east-1 region."},
 'azure': {'instance': {'cloud_provider': 'Azure',
   'name': 'B1s',
   'cpu': 1,
   'ram': 1,
   'storage': 30,
   'gpu': 'None'},
  'power_consumption': {'monthly': '10.44 kWh', 'hourly': '0.014 kWh'},
  'carbon_footprint': {'monthly': '4.5 kg CO2', 'hourly': '0.006 kg CO2'},
  'description': "On Azure, the B1s instance appears suitable for this application. It comes with 1 vCPU and 1 GB of RAM. We'll assume 30GB storage is sufficient.  The power consumption and carbon footprint figures are based on Azure's sustainability calculator, assuming continuous operation in the East US region."},
 'conclusion': {'instance': {'cloud_provider': 'AWS',
   'name': 't4g.nano',
   'cpu': 2,
   'ram': 0.5,
   'storage': 30,
   'gpu': 'None'},
  'power_consumption': {'monthly': '8.76 kWh', 'hourly': '0.012 kWh'},
  'carbon_footprint': {'monthly': '3.77 kg CO2', 'hourly': '0.005 kg CO2'},
  'description': "Based on the analysis, while all three cloud providers offer suitable instances for this TODO application, AWS's t4g.nano instance emerges as the most efficient option. It offers a balance of performance and low resource consumption, resulting in the lowest estimated power consumption and carbon footprint."}}

In [12]:
import math
from google.cloud.firestore_v1.base_query import FieldFilter, Or, And

vendor = gemini_result["aws"]["instance"]["cloud_provider"]
name = gemini_result["aws"]["instance"]["name"]
cpu = gemini_result["aws"]["instance"]["cpu"]
ram = gemini_result["aws"]["instance"]["ram"]
ref = db.collection("cloud_cost")
vendor_filter = FieldFilter("vendor", "==", vendor)
name_filter = FieldFilter("name", "==", name)
cpu_filter = FieldFilter("cpu", "==", cpu)
ram_filter = FieldFilter("ram", "==", ram)
resource_filter = And(filters=[cpu_filter, ram_filter])
instance_filter = Or(filters=[name_filter, resource_filter])
final_filter = And(filters=[vendor_filter, instance_filter])
docs = ref.where(filter=final_filter).stream()

lowest_instance = {"cost_per_hour": float("inf")}
for doc in docs:
    if lowest_instance["cost_per_hour"] > doc.to_dict()["cost_per_hour"]:
        lowest_instance = doc.to_dict()
lowest_instance

{'cpu': 2,
 'ram': 0.5,
 'cost_per_hour': 0.0042,
 'region': 'us-east-1',
 'gpu': None,
 'vendor': 'AWS',
 'name': 't4g.nano'}